In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from io import StringIO
import matplotlib.dates as dates
from datetime import datetime
import matplotlib.ticker as ticker
from random import shuffle
from time import time
from pandas.plotting import register_matplotlib_converters
%matplotlib notebook

#os.chdir('C:\\Users\\A Sua\\Documents\\FIU\\CAP2020')
os.chdir('C:\\Users\\Daniel\\Python and R Notebooks\\CAP2020\\CAP2020')

## Define Functions

In [13]:
# change to directory of raw 765 quarterly files
folder = "C:\\Users\\Daniel\\Python and R Notebooks\\CAP2020\\CAP2020\\data\\raw\\stock_quarterly_data"
interim_data = "data\\interim\\interim_quarterly_roe.csv"

In [14]:
#print computation time script
def timing(f):
    def wrap(*args):
        time1 = time()
        ret = f(*args)
        time2 = time()
        print('{:s} function took {:.3f} ms'.format(f.__name__, (time2-time1)*1000.0))

        return ret
    return wrap

## Import Data

In [15]:
data = []
for file in Path(folder).glob('*.csv'):
        df = pd.read_csv(file)
        stock_name = os.path.basename(file)
        stock_name_noext = stock_name.replace("_quarterly_financial_data.csv", "")
        df['Symbol'] = stock_name_noext
        data.append(df)

In [16]:
bigframe = pd.concat(data)
print('initial shape {}'.format(bigframe.shape))

initial shape (62253, 42)


## Begin Cleaning

### Drop duplicates

In [17]:
bigframe = bigframe.drop_duplicates()
print(print('new shape: {}'.format(bigframe.shape)))

new shape: (62253, 42)
None


### Replace "None" with NaN

In [18]:
bigframe.replace('None', np.nan, inplace=True)

### Clean Ticker Names

In [19]:
bigframe['Symbol'] = bigframe['Symbol'].astype(str)

In [20]:
# replace ABI_0 with ABI
bigframe['Symbol'].replace({'ABI_0':'ABI'}, inplace = True)

# replace CCE_1 with CCE
bigframe['Symbol'].replace({'CCE_1':'CCE'}, inplace = True)

# DG_1 with DG
bigframe['Symbol'].replace({'DG_1':'DG'}, inplace = True)

# replace GAS_0 with GAS_new
bigframe['Symbol'].replace({'GAS_0':'GAS_new'}, inplace = True)

# replace GAS with GAS_old
bigframe['Symbol'].replace({'GAS':'GAS_old'}, inplace = True)

# replace GCI_0 with TEGNA
bigframe['Symbol'].replace({'GCI_0':'TEGNA'}, inplace = True)

#replace GGP_0 with GGP
bigframe['Symbol'].replace({'GGP_0':'GGP'}, inplace = True)

#replace GM_0 with GM
bigframe['Symbol'].replace({'GM_0':'GM'}, inplace = True)

#replace LEH_0 with LEH
bigframe['Symbol'].replace({'LEH_0':'LEH'}, inplace = True)

#replace MNST_0 with MWW
bigframe['Symbol'].replace({'MNST_0':'MWW'}, inplace = True)

# replace Q_1 with IQV
bigframe['Symbol'].replace({'Q_1':'IQV'}, inplace = True)

# replace WM_0 with WM_old
bigframe['Symbol'].replace({'WM_0':'WM_old'}, inplace = True)

#replace PLD with PLD_old
bigframe['Symbol'].replace({'PLD':'PLD_old'}, inplace = True)

#replace PLD_0 with PLD_new
bigframe['Symbol'].replace({'PLD_0':'PLD_new'}, inplace = True)

### Transform column into float

In [21]:
bigframe['ROE'] = bigframe['ROE'].astype(float)

### Quarter End

In [22]:
bigframe['Quarter end'] = pd.to_datetime(bigframe['Quarter end'], errors='ignore')
bigframe

,Quarter end,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,Shareholders equity,Non-controlling interest,...,P/E ratio,Cumulative dividends per share,Dividend payout ratio,Long-term debt to equity ratio,Equity to assets ratio,Net margin,Asset turnover,Free cash flow per share,Current ratio,Symbol
0,2019-09-30,67151778,67151778,1.0,3219620000,NaN,1351965000,NaN,1867655000,0,...,21.69,1.040,0.0488,0.1859,0.5801,0.0509,1.26,1.32,NaN,AAN
1,2019-06-30,67547497,67547497,1.0,3180198000,NaN,1330640000,NaN,1849558000,0,...,19.94,1.010,0.0464,0.188,0.5816,0.052,1.31,0.84,NaN,AAN
2,2019-03-31,67677449,67677449,1.0,3237966000,NaN,1424458000,NaN,1813508000,0,...,17.49,0.970,0.0459,0.2251,0.5601,0.0515,1.37,2.09,NaN,AAN
3,2018-12-31,67202919,67202919,1.0,2826692000,NaN,1065984000,NaN,1760708000,0,...,10.83,0.940,0.0455,0.1937,0.6229,0.0512,1.41,-0.43,NaN,AAN
4,2018-09-30,68594784,68594784,1.0,2698898000,NaN,935392000,NaN,1763506000,0,...,12,0.900,0.0267,0.1686,0.6534,0.0839,1.39,1.13,NaN,AAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,2014-03-30,501018690,501018690,1.0,6403000000,3222000000,5304000000,1127000000,1077000000,22000000,...,30.45,0.340,0.2623,3.3816,0.1682,0.1136,0.72,-0.14,2.8589,ZTS
21,2013-12-31,500729429,500729429,1.0,6558000000,3357000000,5596000000,1415000000,940000000,22000000,...,NaN,0.270,0.263,3.8745,0.1433,NaN,NaN,0.51,2.3724,ZTS
22,2013-09-29,500006833,500006833,1.0,6259000000,3083000000,5393000000,1308000000,843000000,23000000,...,NaN,0.200,NaN,4.3203,0.1347,NaN,NaN,0.12,2.357,ZTS
23,2013-06-30,500000000,500000000,1.0,6261000000,3082000000,5438000000,1321000000,800000000,23000000,...,NaN,0.130,NaN,4.55,0.1278,NaN,NaN,-0.13,2.3331,ZTS


In [23]:
finalframe = bigframe[['Quarter end', 'ROE', 'Symbol']]
finalframe

,Quarter end,ROE,Symbol
0,2019-09-30,0.1099,AAN
1,2019-06-30,0.1136,AAN
2,2019-03-31,0.1130,AAN
3,2018-12-31,0.1117,AAN
4,2018-09-30,0.1785,AAN
...,...,...,...
20,2014-03-30,0.5672,ZTS
21,2013-12-31,NaN,ZTS
22,2013-09-29,NaN,ZTS
23,2013-06-30,NaN,ZTS


## Convert Dates to Quarters

In [24]:
finalframe['Quarter end'] = pd.to_datetime(finalframe['Quarter end'], infer_datetime_format = True)
finalframe.dtypes

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Quarter end    datetime64[ns]
ROE                   float64
Symbol                 object
dtype: object

In [25]:
# period filters
periods_2 = [(finalframe['Quarter end'] >='10/22/2019'), 
            (finalframe['Quarter end'] < '10/22/2019') & (finalframe['Quarter end'] >= '7/22/2019'),
            (finalframe['Quarter end'] < '7/22/2019') & (finalframe['Quarter end'] >= '4/25/2019'),
            (finalframe['Quarter end'] < '4/25/2019') & (finalframe['Quarter end'] >= '1/22/2019'),
            (finalframe['Quarter end'] < '1/22/2019') & (finalframe['Quarter end'] >= '10/22/2018'),
            (finalframe['Quarter end'] < '10/22/2018') & (finalframe['Quarter end'] >= '7/22/2018'),
            (finalframe['Quarter end'] < '7/22/2018') & (finalframe['Quarter end'] >= '4/25/2018'),
            (finalframe['Quarter end'] < '4/25/2018') & (finalframe['Quarter end'] >= '1/22/2018'),
            (finalframe['Quarter end'] < '1/22/2018') & (finalframe['Quarter end'] >= '10/22/2017'),
            (finalframe['Quarter end'] < '10/22/2017') & (finalframe['Quarter end'] >= '7/22/2017'),
            (finalframe['Quarter end'] < '7/22/2017') & (finalframe['Quarter end'] >= '4/25/2017'),
            (finalframe['Quarter end'] < '4/25/2017') & (finalframe['Quarter end'] >= '1/22/2017'),
            (finalframe['Quarter end'] < '1/22/2017') & (finalframe['Quarter end'] >= '10/22/2016'),
            (finalframe['Quarter end'] < '10/22/2016') & (finalframe['Quarter end'] >= '7/22/2016'),
            (finalframe['Quarter end'] < '7/22/2016') & (finalframe['Quarter end'] >= '4/25/2016'),
            (finalframe['Quarter end'] < '4/25/2016') & (finalframe['Quarter end'] >= '1/22/2016'),
            (finalframe['Quarter end'] < '1/22/2016') & (finalframe['Quarter end'] >= '10/22/2015'),
            (finalframe['Quarter end'] < '10/22/2015') & (finalframe['Quarter end'] >= '7/22/2015'),
            (finalframe['Quarter end'] < '7/22/2015') & (finalframe['Quarter end'] >= '4/25/2015'),
            (finalframe['Quarter end'] < '4/25/2015') & (finalframe['Quarter end'] >= '1/22/2015'),
            (finalframe['Quarter end'] < '1/22/2015') & (finalframe['Quarter end'] >= '10/22/2014'),
            (finalframe['Quarter end'] < '10/22/2014') & (finalframe['Quarter end'] >= '7/22/2014'),
            (finalframe['Quarter end'] < '7/22/2014') & (finalframe['Quarter end'] >= '4/25/2014'),
            (finalframe['Quarter end'] < '4/25/2014') & (finalframe['Quarter end'] >= '1/22/2014'),
            (finalframe['Quarter end'] < '1/22/2014') & (finalframe['Quarter end'] >= '10/22/2013'),
            (finalframe['Quarter end'] < '10/22/2013') & (finalframe['Quarter end'] >= '7/22/2013'),
            (finalframe['Quarter end'] < '7/22/2013') & (finalframe['Quarter end'] >= '4/25/2013'),
            (finalframe['Quarter end'] < '4/25/2013') & (finalframe['Quarter end'] >= '1/22/2013'),
            (finalframe['Quarter end'] < '1/22/2013') & (finalframe['Quarter end'] >= '10/22/2012'),
            (finalframe['Quarter end'] < '10/22/2012') & (finalframe['Quarter end'] >= '7/22/2012'),
            (finalframe['Quarter end'] < '7/22/2012') & (finalframe['Quarter end'] >= '4/25/2012'),
            (finalframe['Quarter end'] < '4/25/2012') & (finalframe['Quarter end'] >= '1/22/2012'),
            (finalframe['Quarter end'] < '1/22/2012') & (finalframe['Quarter end'] >= '10/22/2011'),
            (finalframe['Quarter end'] < '10/22/2011') & (finalframe['Quarter end'] >= '7/22/2011'),
            (finalframe['Quarter end'] < '7/22/2011') & (finalframe['Quarter end'] >= '4/25/2011'),
            (finalframe['Quarter end'] < '4/25/2011') & (finalframe['Quarter end'] >= '1/22/2011'),
            (finalframe['Quarter end'] < '1/22/2011') & (finalframe['Quarter end'] >= '10/22/2010'),
            (finalframe['Quarter end'] < '10/22/2010') & (finalframe['Quarter end'] >= '7/22/2010'),
            (finalframe['Quarter end'] < '7/22/2010') & (finalframe['Quarter end'] >= '4/25/2010'),
            (finalframe['Quarter end'] < '4/25/2010') & (finalframe['Quarter end'] >= '1/22/2010'),
            (finalframe['Quarter end'] < '1/22/2010') & (finalframe['Quarter end'] >= '10/22/2009'),
            (finalframe['Quarter end'] < '10/22/2009') & (finalframe['Quarter end'] >= '7/22/2009'),
            (finalframe['Quarter end'] < '7/22/2009') & (finalframe['Quarter end'] >= '4/25/2009'),
            (finalframe['Quarter end'] < '4/25/2009') & (finalframe['Quarter end'] >= '1/22/2009'),
            (finalframe['Quarter end'] < '1/22/2009') & (finalframe['Quarter end'] >= '10/22/2008'),
            (finalframe['Quarter end'] < '10/22/2008') & (finalframe['Quarter end'] >= '7/22/2008'),
            (finalframe['Quarter end'] < '7/22/2008') & (finalframe['Quarter end'] >= '4/25/2008'),
            (finalframe['Quarter end'] < '4/25/2008') & (finalframe['Quarter end'] >= '1/22/2008'),
            (finalframe['Quarter end'] < '1/22/2008') & (finalframe['Quarter end'] >= '10/22/2007'),
            (finalframe['Quarter end'] < '10/22/2007') & (finalframe['Quarter end'] >= '7/22/2007'),
            (finalframe['Quarter end'] < '7/22/2007') & (finalframe['Quarter end'] >= '4/25/2007'),
            (finalframe['Quarter end'] < '4/25/2007') & (finalframe['Quarter end'] >= '1/22/2007'),
            (finalframe['Quarter end'] < '1/22/2007') & (finalframe['Quarter end'] >= '10/22/2006'),
            (finalframe['Quarter end'] < '10/22/2006') & (finalframe['Quarter end'] >= '7/22/2006'),
            (finalframe['Quarter end'] < '7/22/2006') & (finalframe['Quarter end'] >= '4/25/2006'),
            (finalframe['Quarter end'] < '4/25/2006') & (finalframe['Quarter end'] >= '1/22/2006'),
            (finalframe['Quarter end'] < '1/22/2006') & (finalframe['Quarter end'] >= '10/22/2005'),
            (finalframe['Quarter end'] < '10/22/2005') & (finalframe['Quarter end'] >= '7/22/2005'),
            (finalframe['Quarter end'] < '7/22/2005') & (finalframe['Quarter end'] >= '4/25/2005'),
            (finalframe['Quarter end'] < '4/25/2005') & (finalframe['Quarter end'] >= '1/22/2005'),
            (finalframe['Quarter end'] < '1/22/2005') & (finalframe['Quarter end'] >= '10/22/2004'),
            (finalframe['Quarter end'] < '10/22/2004') & (finalframe['Quarter end'] >= '7/22/2004'),
            (finalframe['Quarter end'] < '7/22/2004') & (finalframe['Quarter end'] >= '4/25/2004'),
            (finalframe['Quarter end'] < '4/25/2004') & (finalframe['Quarter end'] >= '1/22/2004'),
            (finalframe['Quarter end'] < '1/22/2004') & (finalframe['Quarter end'] >= '10/22/2003'),
            (finalframe['Quarter end'] < '10/22/2003') & (finalframe['Quarter end'] >= '7/22/2003'),
            (finalframe['Quarter end'] < '7/22/2003') & (finalframe['Quarter end'] >= '4/25/2003'),
            (finalframe['Quarter end'] < '4/25/2003') & (finalframe['Quarter end'] >= '1/22/2003'),
            (finalframe['Quarter end'] < '1/22/2003') & (finalframe['Quarter end'] >= '10/22/2002'),
            (finalframe['Quarter end'] < '10/22/2002') & (finalframe['Quarter end'] >= '7/22/2002'),
            (finalframe['Quarter end'] < '7/22/2002') & (finalframe['Quarter end'] >= '4/25/2002'),
            (finalframe['Quarter end'] < '4/25/2002') & (finalframe['Quarter end'] >= '1/22/2002'),
            (finalframe['Quarter end'] < '1/22/2002') & (finalframe['Quarter end'] >= '10/22/2001'),
            (finalframe['Quarter end'] < '10/22/2001') & (finalframe['Quarter end'] >= '7/22/2001'),
            (finalframe['Quarter end'] < '7/22/2001') & (finalframe['Quarter end'] >= '4/25/2001'),
            (finalframe['Quarter end'] < '4/25/2001') & (finalframe['Quarter end'] >= '1/22/2001'),
            (finalframe['Quarter end'] < '1/22/2001') & (finalframe['Quarter end'] >= '10/22/2000'),
            (finalframe['Quarter end'] < '10/22/2000') & (finalframe['Quarter end'] >= '7/22/2000'),
            (finalframe['Quarter end'] < '7/22/2000') & (finalframe['Quarter end'] >= '4/25/2000'),
            (finalframe['Quarter end'] < '4/25/2000') & (finalframe['Quarter end'] >= '1/22/2000'),
            (finalframe['Quarter end'] < '1/22/2000') & (finalframe['Quarter end'] >= '10/22/1999'),
            (finalframe['Quarter end'] < '10/22/1999') & (finalframe['Quarter end'] >= '7/22/1999'),
            (finalframe['Quarter end'] < '7/22/1999') & (finalframe['Quarter end'] >= '4/25/1999'),
            (finalframe['Quarter end'] < '4/25/1999') & (finalframe['Quarter end'] >= '1/22/1999'),
            (finalframe['Quarter end'] < '1/22/1999') & (finalframe['Quarter end'] >= '10/22/1998'),
            (finalframe['Quarter end'] < '10/22/1998') & (finalframe['Quarter end'] >= '7/22/1998'),
            (finalframe['Quarter end'] < '7/22/1998') & (finalframe['Quarter end'] >= '4/25/1998'),
            (finalframe['Quarter end'] < '4/25/1998') & (finalframe['Quarter end'] >= '1/22/1998'),
            (finalframe['Quarter end'] < '1/22/1998') & (finalframe['Quarter end'] >= '10/22/1997'),
            (finalframe['Quarter end'] < '10/22/1997') & (finalframe['Quarter end'] >= '7/22/1997'),
            (finalframe['Quarter end'] < '7/22/1997') & (finalframe['Quarter end'] >= '4/25/1997'),
            (finalframe['Quarter end'] < '4/25/1997') & (finalframe['Quarter end'] >= '1/22/1997'),
            (finalframe['Quarter end'] < '1/22/1997') & (finalframe['Quarter end'] >= '10/22/1996'),
            (finalframe['Quarter end'] < '10/22/1996') & (finalframe['Quarter end'] >= '7/22/1996'),
            (finalframe['Quarter end'] < '7/22/1996') & (finalframe['Quarter end'] >= '4/25/1996'),
            (finalframe['Quarter end'] < '4/25/1996') & (finalframe['Quarter end'] >= '1/22/1996'),
            (finalframe['Quarter end'] < '1/22/1996') & (finalframe['Quarter end'] >= '10/22/1995'),
            (finalframe['Quarter end'] < '10/22/1995') & (finalframe['Quarter end'] >= '7/22/1995'),
            (finalframe['Quarter end'] < '7/22/1995') & (finalframe['Quarter end'] >= '4/25/1995'),
            (finalframe['Quarter end'] < '4/25/1995') & (finalframe['Quarter end'] >= '1/22/1995'),
            (finalframe['Quarter end'] < '1/22/1995') & (finalframe['Quarter end'] >= '10/22/1994'),
            (finalframe['Quarter end'] < '10/22/1994') & (finalframe['Quarter end'] >= '7/22/1994'),
            (finalframe['Quarter end'] < '7/22/1994') & (finalframe['Quarter end'] >= '4/25/1994'),
            (finalframe['Quarter end'] < '4/25/1994') & (finalframe['Quarter end'] >= '1/22/1994'),
            (finalframe['Quarter end'] < '1/22/1994') & (finalframe['Quarter end'] >= '10/22/1993'),
            (finalframe['Quarter end'] < '10/22/1993') & (finalframe['Quarter end'] >= '7/22/1993'),
            (finalframe['Quarter end'] < '7/22/1993') & (finalframe['Quarter end'] >= '4/25/1993')]

quarters_2 = ['2019Q4', '2019Q3', '2019Q2', '2019Q1', '2018Q4', '2018Q3', '2018Q2', '2018Q1',
            '2017Q4', '2017Q3', '2017Q2', '2017Q1', '2016Q4', '2016Q3', '2016Q2', '2016Q1',
            '2015Q4', '2015Q3', '2015Q2', '2015Q1', '2014Q4', '2014Q3', '2014Q2', '2014Q1',
            '2013Q4', '2013Q3', '2013Q2', '2013Q1', '2012Q4', '2012Q3', '2012Q2', '2012Q1',
            '2011Q4', '2011Q3', '2011Q2', '2011Q1', '2010Q4', '2010Q3', '2010Q2', '2010Q1',
            '2009Q4', '2009Q3', '2009Q2', '2009Q1', '2008Q4', '2008Q3', '2008Q2', '2008Q1',
            '2007Q4', '2007Q3', '2007Q2', '2007Q1', '2006Q4', '2006Q3', '2006Q2', '2006Q1',
            '2005Q4', '2005Q3', '2005Q2', '2005Q1', '2004Q4', '2004Q3', '2004Q2', '2004Q1',
            '2003Q4', '2003Q3', '2003Q2', '2003Q1', '2002Q4', '2002Q3', '2002Q2', '2002Q1',
            '2001Q4', '2001Q3', '2001Q2', '2001Q1', '2000Q4', '2000Q3', '2000Q2', '2000Q1',
            '1999Q4', '1999Q3', '1999Q2', '1999Q1', '1998Q4', '1998Q3', '1998Q2', '1998Q1',
            '1997Q4', '1997Q3', '1997Q2', '1997Q1', '1996Q4', '1996Q3', '1996Q2', '1996Q1',
            '1995Q4', '1995Q3', '1995Q2', '1995Q1', '1994Q4', '1994Q3', '1994Q2', '1994Q1',
            '1993Q4', '1993Q3', '1993Q2']

finalframe['Quarter'] = np.select(periods_2, quarters_2)
finalframe

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Quarter end,ROE,Symbol,Quarter
0,2019-09-30,0.1099,AAN,2019Q3
1,2019-06-30,0.1136,AAN,2019Q2
2,2019-03-31,0.1130,AAN,2019Q1
3,2018-12-31,0.1117,AAN,2018Q4
4,2018-09-30,0.1785,AAN,2018Q3
...,...,...,...,...
20,2014-03-30,0.5672,ZTS,2014Q1
21,2013-12-31,NaN,ZTS,2013Q4
22,2013-09-29,NaN,ZTS,2013Q3
23,2013-06-30,NaN,ZTS,2013Q2


In [ ]:
df2 = finalframe[finalframe.duplicated(['Symbol', 'Quarter'], keep = False)]
df2['Symbol'].unique()

## Export to .csv for edits in Excel

In [ ]:
finalframe.to_csv(interim_data, index = False)

## Import Edited File

In [2]:
path = "data\\interim\\interim_quarterly_roe_edited.csv"
df_roe = pd.read_csv(path)
df_roe.index = df_roe['Quarter end']
df_roe = df_roe.drop('Quarter end', axis=1)
df_roe.index.names=['qend']
df_roe.index = pd.to_datetime(df_roe.index)

In [3]:
df_roe

,ROE,Symbol,Quarter
qend,,,
2019-09-30,0.1099,AAN,2019Q3
2019-06-30,0.1136,AAN,2019Q2
2019-03-31,0.1130,AAN,2019Q1
2018-12-31,0.1117,AAN,2018Q4
2018-09-30,0.1785,AAN,2018Q3
...,...,...,...
2014-03-30,0.5672,ZTS,2014Q1
2013-12-31,NaN,ZTS,2013Q4
2013-09-29,NaN,ZTS,2013Q3


In [4]:
print('old shape {}'.format(df_roe.shape))
df_roe = df_roe.drop_duplicates()
print('new shape: {}'.format(df_roe.shape))

old shape (62251, 3)
new shape: (62251, 3)


In [5]:
df_roe.columns = ['ROE', 'symbol', 'quarter']

## Change Datatypes

In [6]:
df_roe['symbol'] = df_roe['symbol'].astype(str)
df_roe['quarter'] = df_roe['quarter'].astype(str)
df_roe['ROE'] = df_roe['ROE'].astype(float)

## Link Quarter to End Dates

In [7]:
df_roe['new_key'] = df_roe['symbol'] + '_'+ df_roe['quarter']

In [8]:
dates_table = pd.read_csv('data\\external\\quarter_symbols.csv')
dates_table['std_date']  = pd.to_datetime(dates_table['std_date'])
dates_table['index'] = dates_table['Symbol'] + '_' + dates_table['Quarter']

In [9]:
dates_table

,Quarter end,Symbol,Quarter,std_date,Unnamed: 4,Unnamed: 5,Unnamed: 6,2006Q4,12/31/2006,index
0,12/31/2018,AAN,2018Q4,2018-12-31,NaN,NaN,NaN,2007Q1,3/31/2007,AAN_2018Q4
1,9/30/2018,AAN,2018Q3,2018-09-30,NaN,NaN,NaN,2007Q2,6/30/2007,AAN_2018Q3
2,6/30/2018,AAN,2018Q2,2018-06-30,NaN,NaN,NaN,2007Q3,9/30/2007,AAN_2018Q2
3,3/31/2018,AAN,2018Q1,2018-03-31,NaN,NaN,NaN,2007Q4,12/31/2007,AAN_2018Q1
4,12/31/2017,AAN,2017Q4,2017-12-31,NaN,NaN,NaN,2008Q1,3/31/2008,AAN_2017Q4
...,...,...,...,...,...,...,...,...,...,...
32001,3/30/2014,ZTS,2014Q1,2014-03-31,NaN,NaN,NaN,NaN,NaN,ZTS_2014Q1
32002,12/31/2013,ZTS,2013Q4,2013-12-31,NaN,NaN,NaN,NaN,NaN,ZTS_2013Q4
32003,9/29/2013,ZTS,2013Q3,2013-09-30,NaN,NaN,NaN,NaN,NaN,ZTS_2013Q3
32004,6/30/2013,ZTS,2013Q2,2013-06-30,NaN,NaN,NaN,NaN,NaN,ZTS_2013Q2


## Filter Out Symbols that only traded for the entirety of 2007-2018

In [10]:
df_07_18 = df_roe[df_roe['quarter'].str.contains('2007') | df_roe['quarter'].str.contains('2008')| df_roe['quarter'].str.contains('2009')|
           df_roe['quarter'].str.contains('2010') | df_roe['quarter'].str.contains('2011')| df_roe['quarter'].str.contains('2012')|
           df_roe['quarter'].str.contains('2013') | df_roe['quarter'].str.contains('2014')| df_roe['quarter'].str.contains('2015')|
           df_roe['quarter'].str.contains('2016') | df_roe['quarter'].str.contains('2017')| df_roe['quarter'].str.contains('2018')]

In [11]:
df_07_18

,ROE,symbol,quarter,new_key
qend,,,,
2018-12-31,0.1117,AAN,2018Q4,AAN_2018Q4
2018-09-30,0.1785,AAN,2018Q3,AAN_2018Q3
2018-06-30,0.1727,AAN,2018Q2,AAN_2018Q2
2018-03-31,0.1766,AAN,2018Q1,AAN_2018Q1
2017-12-31,0.1844,AAN,2017Q4,AAN_2017Q4
...,...,...,...,...
2014-03-30,0.5672,ZTS,2014Q1,ZTS_2014Q1
2013-12-31,NaN,ZTS,2013Q4,ZTS_2013Q4
2013-09-29,NaN,ZTS,2013Q3,ZTS_2013Q3


In [12]:
df_merge = pd.merge(dates_table, df_roe, left_on='index', right_on='new_key', how='inner')

In [13]:
df_merge

,Quarter end,Symbol,Quarter,std_date,Unnamed: 4,Unnamed: 5,Unnamed: 6,2006Q4,12/31/2006,index,ROE,symbol,quarter,new_key
0,12/31/2018,AAN,2018Q4,2018-12-31,NaN,NaN,NaN,2007Q1,3/31/2007,AAN_2018Q4,0.1117,AAN,2018Q4,AAN_2018Q4
1,9/30/2018,AAN,2018Q3,2018-09-30,NaN,NaN,NaN,2007Q2,6/30/2007,AAN_2018Q3,0.1785,AAN,2018Q3,AAN_2018Q3
2,6/30/2018,AAN,2018Q2,2018-06-30,NaN,NaN,NaN,2007Q3,9/30/2007,AAN_2018Q2,0.1727,AAN,2018Q2,AAN_2018Q2
3,3/31/2018,AAN,2018Q1,2018-03-31,NaN,NaN,NaN,2007Q4,12/31/2007,AAN_2018Q1,0.1766,AAN,2018Q1,AAN_2018Q1
4,12/31/2017,AAN,2017Q4,2017-12-31,NaN,NaN,NaN,2008Q1,3/31/2008,AAN_2017Q4,0.1844,AAN,2017Q4,AAN_2017Q4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32000,3/30/2014,ZTS,2014Q1,2014-03-31,NaN,NaN,NaN,NaN,NaN,ZTS_2014Q1,0.5672,ZTS,2014Q1,ZTS_2014Q1
32001,12/31/2013,ZTS,2013Q4,2013-12-31,NaN,NaN,NaN,NaN,NaN,ZTS_2013Q4,NaN,ZTS,2013Q4,ZTS_2013Q4
32002,9/29/2013,ZTS,2013Q3,2013-09-30,NaN,NaN,NaN,NaN,NaN,ZTS_2013Q3,NaN,ZTS,2013Q3,ZTS_2013Q3
32003,6/30/2013,ZTS,2013Q2,2013-06-30,NaN,NaN,NaN,NaN,NaN,ZTS_2013Q2,NaN,ZTS,2013Q2,ZTS_2013Q2


In [14]:
symbol_list = df_merge['symbol'].unique()

## ROE Table

In [15]:
df_reduced = df_merge[['std_date', 'symbol', 'ROE']]

In [16]:
df2 = df_reduced[df_reduced.duplicated(['symbol', 'std_date'], keep = False)]
df2['symbol'].unique()

array([], dtype=object)

In [17]:
df_reduced = df_merge[['std_date', 'symbol', 'ROE']]
########----------------#######
df_newq = pd.DataFrame([])

for symbol in symbol_list:
    df_temp = df_reduced[df_reduced['symbol']==symbol]
    df_temp.index = df_temp['std_date']
    df_temp = df_temp[['ROE']]
    df_temp = df_temp.T
    df_temp.index = [symbol]
    df_newq = pd.concat([df_newq, df_temp], axis=0)

df_newq

std_date,2006-12-31,2007-03-31,2007-06-30,2007-09-30,2007-12-31,2008-03-31,2008-06-30,2008-09-30,2008-12-31,2009-03-31,...,2016-12-31,2017-03-31,2017-06-30,2017-09-30,2017-12-31,2018-03-31,2018-06-30,2018-09-30,2018-12-31,2019-03-31
AAN,NaN,0.1441,0.1375,0.1307,0.1220,0.1128,0.1155,0.1200,0.1241,0.1334,...,0.0958,0.0969,0.0941,0.0896,0.1844,0.1766,0.1727,0.1785,0.1117,NaN
AAPL,NaN,0.2605,0.2674,0.2719,0.2858,0.2770,0.2667,0.2561,0.2381,0.2284,...,0.3494,0.3509,0.3540,0.3629,0.3737,0.3997,0.4350,0.4868,0.5092,NaN
AAP,NaN,0.2292,0.2195,0.2159,0.2157,0.2298,0.2464,0.2441,0.2310,0.2294,...,0.1649,0.1416,0.1245,0.1149,0.1486,0.1514,0.1545,0.1554,0.1176,NaN
AA,NaN,0.1562,0.1482,0.1464,0.1603,0.1371,0.1268,0.1104,-0.0050,-0.0627,...,-0.0980,-0.0815,-0.0899,-0.1222,-0.0232,-0.0536,-0.0690,-0.0594,0.1208,NaN
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.9272,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YHOO,NaN,0.0845,0.0834,0.0798,0.0721,0.1107,0.1006,0.0862,0.0377,0.0364,...,-0.0070,-0.0005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YUM,NaN,0.6026,0.6126,0.6303,0.6639,0.8815,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZION,NaN,0.1231,0.1204,0.1126,0.0912,0.0823,0.0658,0.0465,-0.0551,-0.2529,...,0.0596,0.0663,0.0747,0.0791,0.0772,0.0913,0.0963,0.1060,0.1209,NaN
ZMH,NaN,0.1748,0.1775,0.1466,0.1466,0.1437,0.1414,0.1696,0.1510,0.1454,...,0.0317,0.0514,0.0722,0.0650,0.1711,0.1520,0.1462,0.1461,-0.0319,NaN


In [18]:
df_newq = df_newq.iloc[:, 1:-1]
df_newq

std_date,2007-03-31,2007-06-30,2007-09-30,2007-12-31,2008-03-31,2008-06-30,2008-09-30,2008-12-31,2009-03-31,2009-06-30,...,2016-09-30,2016-12-31,2017-03-31,2017-06-30,2017-09-30,2017-12-31,2018-03-31,2018-06-30,2018-09-30,2018-12-31
AAN,0.1441,0.1375,0.1307,0.1220,0.1128,0.1155,0.1200,0.1241,0.1334,0.1339,...,0.0978,0.0958,0.0969,0.0941,0.0896,0.1844,0.1766,0.1727,0.1785,0.1117
AAPL,0.2605,0.2674,0.2719,0.2858,0.2770,0.2667,0.2561,0.2381,0.2284,0.2199,...,0.3559,0.3494,0.3509,0.3540,0.3629,0.3737,0.3997,0.4350,0.4868,0.5092
AAP,0.2292,0.2195,0.2159,0.2157,0.2298,0.2464,0.2441,0.2310,0.2294,0.2225,...,0.1691,0.1649,0.1416,0.1245,0.1149,0.1486,0.1514,0.1545,0.1554,0.1176
AA,0.1562,0.1482,0.1464,0.1603,0.1371,0.1268,0.1104,-0.0050,-0.0627,-0.1446,...,-0.0377,-0.0980,-0.0815,-0.0899,-0.1222,-0.0232,-0.0536,-0.0690,-0.0594,0.1208
ABBV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.9272,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YHOO,0.0845,0.0834,0.0798,0.0721,0.1107,0.1006,0.0862,0.0377,0.0364,0.0371,...,-0.1594,-0.0070,-0.0005,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YUM,0.6026,0.6126,0.6303,0.6639,0.8815,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZION,0.1231,0.1204,0.1126,0.0912,0.0823,0.0658,0.0465,-0.0551,-0.2529,-0.2869,...,0.0547,0.0596,0.0663,0.0747,0.0791,0.0772,0.0913,0.0963,0.1060,0.1209
ZMH,0.1748,0.1775,0.1466,0.1466,0.1437,0.1414,0.1696,0.1510,0.1454,0.1404,...,0.0349,0.0317,0.0514,0.0722,0.0650,0.1711,0.1520,0.1462,0.1461,-0.0319


In [19]:
new_symbols_div = []
null_symbols_div = []
for symbol in df_newq.index:
    temp_value = df_newq.loc[symbol, :].isnull().sum()
    if temp_value == 0:
        new_symbols_div.append(symbol)
    else:
        null_symbols_div.append(symbol)

In [20]:
len(new_symbols_div), len(null_symbols_div)

(426, 335)

In [21]:
df_quarter_roe = pd.DataFrame([])
for symbol in new_symbols_div:
    temp_df = df_newq[df_newq.index == symbol]
    df_quarter_roe = pd.concat([df_quarter_roe, temp_df])

df_quarter_roe.shape

(426, 48)

In [22]:
export_path = 'data\\processed\\df_quarter_roe_07_18.csv'
df_quarter_roe.to_csv(export_path)